In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/fines.csv')
df

,CarNumber,Refund,Fines,Make,Model,Year
0,Y163O8161RUS,2,3200.0,Ford,Focus,1989
1,E432XX77RUS,1,6500.0,Toyota,Camry,1995
2,7184TT36RUS,1,2100.0,Ford,Focus,1984
3,X582HE161RUS,2,2000.0,Ford,Focus,2015
4,92918M178RUS,1,5700.0,Ford,Focus,2014
...,...,...,...,...,...,...
930,NaN,1,8594.6,Ford,Focus,1989
931,NaN,1,500.0,Ford,Focus,2001
932,NaN,1,10400.0,Ford,Focus,2004
933,NaN,2,2700.0,BMW,Focus,2012


## Во всех следующих подзадачах нужно рассчитать штрафы/возврат*год для каждой строкй строки, создать новый столбец с рассчитанными данными и измерить время с помощью timeit в ячейке

Сначала рассчитаем обычным циклом с методом append()

In [3]:
%%time
new_col = []

for i in range(0, len(df)):
    new_col.append(df.iloc[i]['Fines'] / (df.iloc[i]['Refund'] * df.iloc[i]['Year']))

df['Fines / Refund * Year'] = new_col

CPU times: user 230 ms, sys: 3.71 ms, total: 234 ms
Wall time: 237 ms


In [4]:
df

,CarNumber,Refund,Fines,Make,Model,Year,Fines / Refund * Year
0,Y163O8161RUS,2,3200.0,Ford,Focus,1989,0.804424
1,E432XX77RUS,1,6500.0,Toyota,Camry,1995,3.258145
2,7184TT36RUS,1,2100.0,Ford,Focus,1984,1.058468
3,X582HE161RUS,2,2000.0,Ford,Focus,2015,0.496278
4,92918M178RUS,1,5700.0,Ford,Focus,2014,2.830189
...,...,...,...,...,...,...,...
930,NaN,1,8594.6,Ford,Focus,1989,4.321066
931,NaN,1,500.0,Ford,Focus,2001,0.249875
932,NaN,1,10400.0,Ford,Focus,2004,5.189621
933,NaN,2,2700.0,BMW,Focus,2012,0.670974


## Метод iterrows()

In [5]:
%%time
new_col = []

for index, row in df.iterrows():
    new_col.append(row['Fines'] / (row['Refund'] * row['Year']))

df['Fines / Refund * Year'] = new_col

CPU times: user 56.2 ms, sys: 2.66 ms, total: 58.8 ms
Wall time: 60.5 ms


## С помощью apply() и лямбда-функции

In [6]:
%%time
df['Fines / Refund * Year'] = df.apply(
    lambda row: row['Fines'] / (row['Refund'] * row['Year']), axis=1
    )

CPU times: user 13.1 ms, sys: 1.33 ms, total: 14.4 ms
Wall time: 13.5 ms


## Используя объекты Series из датафрейма

In [7]:
%%time
df['Fines / Refund * Year'] = df['Fines'] / (df['Refund'] * df['Year'])

CPU times: user 899 µs, sys: 100 µs, total: 999 µs
Wall time: 969 µs


In [8]:
%%time
df['Fines / Refund * Year'] = df['Fines'].values / (
    df['Refund'].values * df['Year'].values
    )

CPU times: user 1.09 ms, sys: 480 µs, total: 1.57 ms
Wall time: 1.32 ms


## Индексация

In [9]:
%%time
df.loc[df['CarNumber'] == 'M0299X197RUS']

CPU times: user 979 µs, sys: 91 µs, total: 1.07 ms
Wall time: 1.01 ms


,CarNumber,Refund,Fines,Make,Model,Year,Fines / Refund * Year
190,M0299X197RUS,2,67200.0,Ford,Focus,1995,16.842105
445,M0299X197RUS,2,19200.0,Ford,Focus,2005,4.788030
473,M0299X197RUS,1,9600.0,Ford,Focus,2007,4.783259
725,M0299X197RUS,2,3000.0,Ford,Focus,1989,0.754148


## Установим CarNumber индекс в вашем фрейме данных с помощью CarNumber

In [10]:
df1 = df.set_index('CarNumber')

## Снова получим строку для того же CarNumber

In [11]:
%%time
df1.loc['M0299X197RUS']

CPU times: user 1.58 ms, sys: 634 µs, total: 2.22 ms
Wall time: 1.67 ms


,Refund,Fines,Make,Model,Year,Fines / Refund * Year
CarNumber,,,,,,
M0299X197RUS,2,67200.0,Ford,Focus,1995,16.842105
M0299X197RUS,2,19200.0,Ford,Focus,2005,4.788030
M0299X197RUS,1,9600.0,Ford,Focus,2007,4.783259
M0299X197RUS,2,3000.0,Ford,Focus,1989,0.754148


## Понижение разряда числовых переменных

In [12]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 935 entries, 0 to 934
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   CarNumber              925 non-null    object 
 1   Refund                 935 non-null    int64  
 2   Fines                  935 non-null    float64
 3   Make                   935 non-null    object 
 4   Model                  924 non-null    object 
 5   Year                   935 non-null    int64  
 6   Fines / Refund * Year  935 non-null    float64
dtypes: float64(2), int64(2), object(3)
memory usage: 204.6 KB


## Создадим оптимизированный датафрейм из исходного. Для этого:

### понизим от float64 до float32 для всех столбцов

### понизим от int64 до наименьшего числового возможного dtype

In [13]:
df_optimized = df.copy()
df_optimized['Fines'] = pd.to_numeric(df_optimized['Fines'], downcast='float')
df_optimized['Refund'] = pd.to_numeric(df_optimized['Refund'], downcast='integer')
df_optimized['Year'] = pd.to_numeric(df_optimized['Year'], downcast='integer')

In [14]:
df_optimized.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 935 entries, 0 to 934
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   CarNumber              925 non-null    object 
 1   Refund                 935 non-null    int8   
 2   Fines                  935 non-null    float32
 3   Make                   935 non-null    object 
 4   Model                  924 non-null    object 
 5   Year                   935 non-null    int16  
 6   Fines / Refund * Year  935 non-null    float64
dtypes: float32(1), float64(1), int16(1), int8(1), object(3)
memory usage: 189.0 KB


## Категории

In [15]:
df_optimized['Make'] = df_optimized['Make'].astype('category')
df_optimized['Model'] = df_optimized['Model'].astype('category')

In [16]:
df_optimized.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 935 entries, 0 to 934
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   CarNumber              925 non-null    object  
 1   Refund                 935 non-null    int8    
 2   Fines                  935 non-null    float32 
 3   Make                   935 non-null    category
 4   Model                  924 non-null    category
 5   Year                   935 non-null    int16   
 6   Fines / Refund * Year  935 non-null    float64 
dtypes: category(2), float32(1), float64(1), int16(1), int8(1), object(1)
memory usage: 79.7 KB


## Видим, что объем занимаемой памяти уменьшился более, чем в два раза

## Очистим память от исходного датафрейма

In [17]:
%reset_selective -f df

In [18]:
df

NameError: name 'df' is not defined

## Видим, что память очистилась